# Who Attended Online User Meetings
This notebook is a supplement to the talk [The Heart of The Algorithm](https://dyalog.tv/APLSeeds23/?v=O73HYH0p8eo) given by Rich Park at the APL Seeds '23 online user meeting. In that talk, he introduces the basic syntax of APL before explaining the core algorithm used to solve this problem.

In this notebook, we outline the problem and show how its solution can be used to gain insights from data. Then, we will build up the solution piece by piece, explaining every part in detail.

## The problem
This problem comes from the 2022 [APL Problem Solving Competition](https://www.dyalog.com/student-competition.htm). We will be solving the first task in problem 3 titled "Meeting of the Minds".

Each year, Dyalog hosts a meeting of staff and users during which we give presentations and share ideas. For 2020 and 2021, these meetings were held online.

Afterwards, we obtained the attendance data from Zoom. It has been simplified and anonymised for the purpose of this problem.

**attendees.csv** is a table with columns:
- Attendee: name of each attendee (fake, anonymous and unique for each attendee)
- Join Time: date and time in `MM/DD/YYYY hh:mm` format when a user joined the meeting
- Leave Time: date and time the user disconnected from the meeting

The same user may join and leave the meeting multiple times.

**schedule.csv** is a table with columns:
- Session: unique text ID for each session
- Title: presentation title or session type (e.g. break)
- Start Time: date time when session started (same format as in attendees.csv)
- End Time: date and time that session ended

I renamed the files to lowercase so that the `]Get` user command brings them in as arrays with the same names used in the problem description. Note that user commands are not valid APL syntax and can only be used in the interactive session (or in notebooks like this) - they cannot be used inside functions.

In [1]:
]Get /d/Presentations/APLSeeds23/attendees.csv
5↑attendees

#.attendees

┌──────────────┬───────────────┬───────────────┬─────────┐
│Attendee │Join Time │Leave Time │Date │
├──────────────┼───────────────┼───────────────┼─────────┤
│Jaslyn Chapman│11/9/2020 15:18│11/9/2020 18:04│11/9/2020│
├──────────────┼───────────────┼───────────────┼─────────┤
│Jaslyn Chapman│11/9/2020 13:57│11/9/2020 15:18│11/9/2020│
├──────────────┼───────────────┼───────────────┼─────────┤
│Dalia Gentry │11/9/2020 13:59│11/9/2020 19:03│11/9/2020│
├──────────────┼───────────────┼───────────────┼─────────┤
│Mark Cannon │11/9/2020 15:42│11/9/2020 16:16│11/9/2020│
└──────────────┴───────────────┴───────────────┴─────────┘

In [2]:
]Get /d/Presentations/APLSeeds23/schedule.csv
5↑schedule

#.schedule

┌────────────┬──────────────────────────────┬───────────────┬───────────────┐
│Session │Title │Start Time │End Time │
├────────────┼──────────────────────────────┼───────────────┼───────────────┤
│2020-D01 │Welcome to Dyalog '20 │11/9/2020 14:00│11/9/2020 14:15│
├────────────┼──────────────────────────────┼───────────────┼───────────────┤
│2020-D02 │The Road Ahead │11/9/2020 14:15│11/9/2020 15:00│
├────────────┼──────────────────────────────┼───────────────┼───────────────┤
│2020-break-1│break │11/9/2020 15:00│11/9/2020 15:15│
├────────────┼──────────────────────────────┼───────────────┼───────────────┤
│2020-D03 │Multi-line Input and Scripting│11/9/2020 15:15│11/9/2020 15:45│
└────────────┴──────────────────────────────┴───────────────┴───────────────┘

## The solution
Here we will define the `Attended` function in order to show a few applications of the result. No code will be explained yet.

In [3]:
∇  Attended←{
     a_cols←1⌷⍺ ⋄ a_data←1↓⍺
     s_cols←1⌷⍵ ⋄ s_data←1↓⍵
     Attendees←{a_data[;a_cols⍳⊆⍵]}
     Schedule←{s_data[;s_cols⍳⊆⍵]}
     Timestamp2Unix←{20 ⎕DT ¯1(⌽@1 2 3)¨2⊃¨': /'∘⎕VFI¨⊆⍵}

     connections←Attendees'Join Time' 'Leave Time'
     connected←'--'∘≢¨connections[;1]

     (join leave)←↓⍉Timestamp2Unix connected⌿connections
     (start end)←↓⍉Timestamp2Unix Schedule'Start Time' 'End Time'

     names←Attendees'Attendee'
     overlap←(connected⌿names){+⌿⍵}⌸⍉0⌈(end∘.⌊leave)-(start∘.⌈join)
     0.5≤(names{∨⌿⍵}⌸connected)⍀overlap÷[2](end-start)
 }
∇

The result is a Boolean matrix of shape $435 \times 48$. This corresponds to $435$ people and $48$ sessions.

In [4]:
⍴att←attendees Attended schedule

435 48

From the result `att`, we can compute:

The number of sessions attended by each person:

In [5]:
+/att

21 23 1 16 0 30 0 0 45 38 39 16 0 0 28 47 4 33 3 19 0 0 0 22 0 0 0 23 0 36 41 8 0 2 23 0 0 0 22 37 23 0 33 4 0 21 26 12 23 47 16 15 11 2 8 23 0 14 48 0 43 23 0 19 22 0 12 37 45 29 10 0 22 1 14 6 6 32 0 0 26 22 9 25 23 34 31 8 8 23 37 0 23 0 18 30 0 16 10 0 36 14 6 23 1 23 4 43 9 22 0 5 24 22 21 26 5 45 0 0 0 0 12 40 42 11 20 0 23 0 23 25 24 19 47 4 47 32 46 7 14 9 21 3 12 0 19 5 23 0 2 0 3 32 14 15 3 32 22 20 0 12 46 8 16 20 46 17 22 21 2 48 0 18 6 14 22 3 10 41 1 22 4 22 0 41 41 16 0 0 23 6 2 10 18 10 14 0 41 16 42 0 35 18 46 0 3 15 9 0 2 2 0 32 0 13 0 44 0 7 0 47 0 0 35 27 15 19 36 18 15 40 16 47 47 2 41 7 30 17 40 18 11 0 21 0 27 23 11 0 23 23 3 12 0 23 47 5 21 48 48 22 17 34 0 0 0 18 16 47 11 0 10 0 10 10 2 6 7 0 0 0 5 7 5 4 4 1 11 7 24 0 0 15 0 12 25 0 17 0 0 1 5 17 2 1 3 5 1 0 2 0 24 4 0 3 5 0 1 24 24 2 5 0 0 1 8 24 0 0 22 0 12 20 0 24 24 9 23 4 0 0 9 0 24 0 7 8 0 11 2 0 1 0 10 3 0 6 1 7 20 0 8 9 0 9 0 0 0 24 0 0 19 25 3 0 7 0 11 12 0 20 25 24 0 13 23 0 14 2 24 24 0 0 10 15 20 0 12 6 7 7 12 6 1 0 15 0 0 24 0 4 14 23 0 18 23 0 0 8 24 16 9 5 22 3 5 1 7 1 12 12 5 0 1

The number of attendees for each session:

In [6]:
+⌿att

160 178 162 169 164 150 157 151 131 134 103 76 160 159 143 143 146 128 128 127 114 114 113 118 129 131 120 120 122 111 112 108 94 95 96 26 112 115 104 109 107 104 102 101 83 84 73 71

If the maximum sessions is equal to the total number of sessions, then that means that some people attended every session:

In [7]:
⌈/+/att   ⍝ Some people attended all 48 sessions

48

If the maximum attendance is equal to the total number of attendees, then some sessions were attended by everyone:

In [8]:
⌈/+⌿att   ⍝ No session was attended by all people

178

Names of attendees who attended all sessions:

In [9]:
(∪1↓attendees[;1])⌿⍨(⊢=⌈/)+/att

┌──────────────┬─────────────┬─────────────────┬──────────────┐
│Jessie Raymond│Judith Phelps│Sharon Stephenson│Garrett Gibson│
└──────────────┴─────────────┴─────────────────┴──────────────┘

Top ten most attended sessions in descending order:

In [10]:
total←+⌿att
order←10↑⍒total
(1↓schedule)[order ; 1 2] , total[order]

┌────────────┬───────────────────────────────────────────────────┬───┐
│2020-D02 │The Road Ahead │178│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D03 │Multi-line Input and Scripting │169│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D04 │The .NET Core Bridge │164│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-break-1│break │162│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D01 │Welcome to Dyalog '20 │160│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D08 │Dyalog's Docker Containers │160│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D09 │Array Notation RC1 │159│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D05 │Time Travel Debugging and Statistical Distributions│157│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-D06 │Reworking Mastering Dyalog APL │151│
├────────────┼───────────────────────────────────────────────────┼───┤
│2020-break-2│break │150│
└────────────┴───────────────────────────────────────────────────┴───┘

Now that we've seen some insights we can gain with our attendance matrix, let's look at how we can actually compute it.

## Selecting data
We can select elements from arrays with square brackets. This is a special syntax in APL, but quite convenient.

In [11]:
schedule[3;2]   ⍝ 3rd row, 2nd column

┌──────────────┐
│The Road Ahead│
└──────────────┘

Omitting an index returns all data along that dimension of the array:

In [12]:
schedule[1;]   ⍝ Header row is the first row

┌───────┬─────┬──────────┬────────┐
│Session│Title│Start Time│End Time│
└───────┴─────┴──────────┴────────┘

It may be preferred to select data from specific columns according to the name of the column from the header row.

To do this, we will to look up the position of our desired column in the header using the **index-of** `⍺⍳⍵` primitive:

In [13]:
schedule[1;]⍳'Start Time'

5 5 5 5 5 5 5 5 5 5

Index-of returns one greater than the length of the left argument (`1+≢⍺`) where elements in the right argument `⍵` are not found. In this case, none of the individual characters in `'Start Time'` were found in our header.

This is because `'Start Time'` has a different structure to our header. `'Start Time'` is a list of characters, whereas our header is a list of lists of characters. It is a nested list of character vectors.

We can see the difference by comparing the `]Box`-ed display:

In [14]:
schedule[1;]    ⍝ nested vector of character vectors
'Start Time'    ⍝ simple character vector
⊂'Start Time'   ⍝ nested scalar containing a character vector

┌───────┬─────┬──────────┬────────┐
│Session│Title│Start Time│End Time│
└───────┴─────┴──────────┴────────┘

Start Time

┌──────────┐
│Start Time│
└──────────┘

Or more formally, using the **depth** `≡⍵` function.

In [15]:
≡schedule[1;]   ⍝ nested vector of character vectors
≡'Start Time'   ⍝ a simple array has depth ≤ 1
≡⊂'Start Time'

2

1

2

The **nest** primitive only encloses its argument if it is simple, which is convenient for cases like this where we may want to supply a single list or a list of lists.

Stranding (juxtaposing arrays with spaces) forms a list of lists:

In [16]:
'one' 'two' 'three'

┌───┬───┬─────┐
│one│two│three│
└───┴───┴─────┘

Enclosing adds a level of nesting:

In [17]:
⊂'one' 'two' 'three'

┌───────────────┐
│┌───┬───┬─────┐│
││one│two│three││
│└───┴───┴─────┘│
└───────────────┘

Enclose-if-simple nests a simple array:

In [18]:
⊆'one'

┌───┐
│one│
└───┘

But adds no extra nesting to an already nested array:

In [19]:
⊆'one' 'two' 'three'

┌───┬───┬─────┐
│one│two│three│
└───┴───┴─────┘

We can now use text column names to select columns from our data. We want the data without the header, so we will drop the first row.

In [20]:
5↑ (1↓schedule)[;schedule[1;]⍳⊆'Start Time' 'Title']

┌───────────────┬──────────────────────────────┐
│11/9/2020 14:00│Welcome to Dyalog '20 │
├───────────────┼──────────────────────────────┤
│11/9/2020 14:15│The Road Ahead │
├───────────────┼──────────────────────────────┤
│11/9/2020 15:00│break │
├───────────────┼──────────────────────────────┤
│11/9/2020 15:15│Multi-line Input and Scripting│
├───────────────┼──────────────────────────────┤
│11/9/2020 15:45│The .NET Core Bridge │
└───────────────┴──────────────────────────────┘

We can factor this out as a function:

In [21]:
Get←{(1↓⍺)[;⍺[1;]⍳⊆⍵]}
5↑ schedule Get 'Start Time' 'Title'

┌───────────────┬──────────────────────────────┐
│11/9/2020 14:00│Welcome to Dyalog '20 │
├───────────────┼──────────────────────────────┤
│11/9/2020 14:15│The Road Ahead │
├───────────────┼──────────────────────────────┤
│11/9/2020 15:00│break │
├───────────────┼──────────────────────────────┤
│11/9/2020 15:15│Multi-line Input and Scripting│
├───────────────┼──────────────────────────────┤
│11/9/2020 15:45│The .NET Core Bridge │
└───────────────┴──────────────────────────────┘

This is very convenient, but `1↓⍺` every time we want some data is a bit expensive. Instead, we will separate the header row on import and refer to the header directly. As an aside, the `⎕CSV` system function can do this while reading the data from file, by giving a `1` as the 4th element of its argument.

In [22]:
path←'/d/Presentations/APLSeeds23/attendees.csv'
(a_data a_cols)←⎕CSV path ⍬ 1 1
Attendees←{a_data[;a_cols⍳⊆⍵]}

path←'/d/Presentations/APLSeeds23/schedule.csv'
(s_data s_cols)←⎕CSV path ⍬ 1 1
Schedule←{s_data[;s_cols⍳⊆⍵]}

## Using datetimes
Our data has datetimes represented as lists of characters. In order to do comparison efficiently, we will convert these into numbers. We will use the Unix time number, which is the number of seconds since 1st January 1970, so that we get 1 second precision.

The system function `⎕VFI` is used to safely convert character data into numbers. The **execute** primitive `⍎⍵` can be used to convert characters into numbers, but because it executes any APL expression it can be dangerous to use with data from external sources.

In the monadic case, `⎕VFI` checks space-separated tokens to see if they are valid APL numeric literals.

In [23]:
⎕VFI'42 1,4 1.5   1e3 2J¯4 2J-4 -6 ¯6'

┌───────────────┬─────────────────────────┐
│1 0 1 1 1 0 0 1│42 0 1.5 1000 2J¯4 0 0 ¯6│
└───────────────┴─────────────────────────┘

It returns a two-element vector. The 1st element is a Boolean mask, a `1` indicates which numbers in the 2nd element were converted from valid literal numbers in the argument. The Boolean mask can be used with **compress** `⍺/⍵` to extract numbers.

The function `⍺/⍵` will replicate elements of `⍵` a number of times specified in `⍺`.

In [24]:
1 0 3 2 / 'ABCD'

ACCCDD

When used with a Boolean `⍺`, it is called *compress* and is often used to select parts of an array.

In [25]:
⊃(//⎕VFI)'42 1,4 1.5   1e3 2J¯4 2J-4 -6 ¯6'

42 1.5 1000 2J¯4 ¯6

We can provide a left argument to specify other separator characters:

In [26]:
'/ :'⎕VFI'11/9/2020 14:00'

┌─────────┬──────────────┐
│1 1 1 1 1│11 9 2020 14 0│
└─────────┴──────────────┘

We know our datetimes should be all numbers, so we'll pick the 2nd element instead of using compress.

In [27]:
2⊃'/ :'⎕VFI'11/9/2020 14:00'

11 9 2020 14 0

The system function `⎕DT` can convert between many datetime formats. We will convert from `⎕TS`-style time stamps to Unix time numbers.

But first, remember that our timestamps have the months first. `⎕TS`-style time stamps are `year month day hour minute second millisecond`. We can omit the milliseconds, but we must rotate our dates to be in the correct order.

In [28]:
¯1⌽11 9 2020

2020 11 9

In a full time stamp, we only want to rotate the first three elements.

In [29]:
¯1(⌽@1 2 3)11 9 2020 14 0

2020 11 9 14 0

We can then turn this into our Unix time number:

In [30]:
20⎕DT⊆2020 11 9 14 0

1604930400

`⎕TS`-style time stamps are numeric lists. `⎕DT` accepts lists of numeric lists, so it can convert many time stamps with a single call. But then we need to enclose a single time stamp so it has the same structure.

In [31]:
2020 11 9 14 0                                     ⍝ A single time stamp
(2020 11 9 14 0)(2020 11 9 14 0)(2020 11 9 14 0)   ⍝ A list of time stamps
⊆2020 11 9 14 0                                    ⍝ A single enclosed timestamp

2020 11 9 14 0

┌──────────────┬──────────────┬──────────────┐
│2020 11 9 14 0│2020 11 9 14 0│2020 11 9 14 0│
└──────────────┴──────────────┴──────────────┘

┌──────────────┐
│2020 11 9 14 0│
└──────────────┘

We can put these pieces together to form our datetime conversion function:

In [32]:
∇ Timestamp2Unix←{
  ⍺←⊢   ⍝ Amount to rotate date. Default is to reverse.
 ⍝ ⍵: character date times 
  20 ⎕DT ⍺∘(⌽@1 2 3)¨2⊃¨'/ :'∘⎕VFI¨⊆⍵
  }
∇

In [33]:
3↑schedule Get 'Start Time' 'End Time'
7↑¯1 Timestamp2Unix schedule Get 'Start Time' 'End Time'

┌───────────────┬───────────────┐
│11/9/2020 14:00│11/9/2020 14:15│
├───────────────┼───────────────┤
│11/9/2020 14:15│11/9/2020 15:00│
├───────────────┼───────────────┤
│11/9/2020 15:00│11/9/2020 15:15│
└───────────────┴───────────────┘

1604930400 1604931300
1604931300 1604934000
1604934000 1604934900
1604934900 1604936700
1604936700 1604938500
1604938500 1604939400
1604939400 1604940900

## Comparing date times
As explained in [the presentation at APL Seeds '23](https://dyalog.tv/APLSeeds23/?v=O73HYH0p8eo), we will define a *connection* using a `join` and a `leave` time, while a *session* will be defined by its `start` and `end` time.

To make our notation more clear, we will assign the vectors of times to variables.

The **split** function `↓⍵` will break up an array into a more nested array by enclosing every row. This is a convenient way to split a matrix of two rows into a 2-element vector of vectors. We can then use **strand assignment** to assign multiple names at once.

In [34]:
(start end) ← ↓⍉ Timestamp2Unix Schedule 'Start Time' 'End Time'

Note the use of **transpose** `⍉⍵`. `Schedule` returned two columns, but `↓⍵` splits rows. Split-transpose `↓⍉⍵` is a commonly-used pattern for taking columns in a matrix and getting a collection of individually named vectors.

If we try the same for `join` and `leave`, we hit an `ERROR`.

In [35]:
(join leave) ← ↓⍉ Timestamp2Unix Attendees 'Join Time' 'Leave Time'

INDEX ERROR
Timestamp2Unix[3] 20 ⎕DT ⍺∘(⌽@1 2 3)¨2⊃¨'/ :'∘⎕VFI¨⊆⍵
                              ∧


Let's looks at rows 6 to 10 of our attendees join and leave times. Here we are using **take** `⍺↑⍵` and **drop** `⍺↓⍵` to quickly select the rows we want to see:

In [36]:
5↑5↓Attendees 'Join Time' 'Leave Time'

┌───────────────┬───────────────┐
│11/9/2020 13:58│11/9/2020 15:16│
├───────────────┼───────────────┤
│-- │-- │
├───────────────┼───────────────┤
│11/9/2020 14:26│11/9/2020 18:09│
├───────────────┼───────────────┤
│-- │-- │
├───────────────┼───────────────┤
│-- │-- │
└───────────────┴───────────────┘

Some rows contain two dashes `'--'` to indicate that a person had registered but not connected that day. We can filter out these rows and replace them after converting date times.

First, use comparison to get a Boolean vector where a `1` indicates a connection (and therefore valid date time to convert):

In [37]:
connections←Attendees 'Join Time' 'Leave Time'
connected←'--'∘≢¨connections[;1]

While equality `⍺=⍵` and inequality `⍺≠⍵` compare simple scalars within two arrays, match `⍺≡⍵` and not match `⍺≢⍵` compare both the contents and the shapes of two arrays to see if they are identical.

In [38]:
((1 2 3) 4 ('ab') 'def') = (1 2 5) 4 ('cb') 'kef'   ⍝ Some items are equal
((1 2 3) 4 ('ab') 'def') ≡ (1 2 5) 4 ('cb') 'kef'   ⍝ Arrays are not identical

┌─────┬─┬───┬─────┐
│1 1 0│1│0 1│0 1 1│
└─────┴─┴───┴─────┘

0

Because we are only looking for a single match, we use the **bind** operator `∘` to "fill" the left slot of the **not match** function `⍺≢⍵`. This forms a monadic function "doesn't match `'--'`". We then apply this function to every element in our `connections` matrix, looping over it with the **each** `¨` operator.

Note that `⎕←` (*quad gets*) is like a *print* statement in some other languages, and lets us assign to a name and see the value in the same line.

In [39]:
⎕←connected←'--'∘≢¨connections[;1]

1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0

We can now use our `connected` Boolean vector to filter out dash `'--'` rows so that we only convert valid date times.

The **replicate-first** `⍺⌿⍵` returns rows of `⍵` duplicated `⍺` times.

In [40]:
⎕←a←3 4⍴⎕A   ⍝ ⎕A is a variable with uppercase alphabet characters
1 0 3 2/a    ⍝ Replicate applies across rows
1 0 3⌿a      ⍝ Replicate-first applies down columns

ABCD
EFGH
IJKL

ACCCDD
EGGGHH
IKKKLL

ABCD
IJKL
IJKL
IJKL

With a Boolean left argument `⍺`, replicate is called **compress**. Its cousin, **expand** `⍺⍀⍵` can be used to add fill data where there are `0`s in `⍺`.

In [41]:
1 1 0 1 ⍀ 'ABC'   ⍝ Put a blank between B and C

AB C

So now we filter out bad datetimes and convert valid datetimes to Unix time numbers.

We will use `connected` later to add blanks to our computed overlaps. If we expand before computing overlaps, our solution would be invalid if there were times around 1st January 1970 in our data.

In [42]:
(join leave)←↓⍉Timestamp2Unix connected⌿connections

Next we define the **overlap** between a particular connection and a particular session. The overlap begins at the *later* of the session `start` and the person `join` times, and ends at the *earlier* time between the session `end` and the person `leave` times.

In [43]:
(end[3] ⌊ leave[7]) - (start[3] ⌈ join[7])

900

We use an outer product `∘.` to compare all `start`, `end`, `join` and `leave` times. We will inspect the top-left 10 by 10 corner.

In [44]:
10 10↑(end∘.⌊leave) - (start∘.⌈join)

¯3780 900 900 ¯5220 ¯3720 900 ¯660 900 840 180
¯1080 2700 2700 ¯2520 ¯1020 2700 2040 2700 2700 ¯720
 ¯180 900 900 ¯1620 ¯120 900 900 900 900 ¯3420
 1620 180 1800 180 1680 60 1800 1800 1800 ¯4320
 1800 ¯1620 1800 1800 1800 ¯1740 1800 1800 1800 ¯6120
 900 ¯3420 900 60 900 ¯3540 900 900 900 ¯7920
 1500 ¯4320 1500 ¯840 1500 ¯4440 1500 1500 1500 ¯8820
 1200 ¯5820 1200 ¯2340 1200 ¯5940 1200 1200 1200 ¯10320
 900 ¯7020 900 ¯3540 240 ¯7140 900 900 900 ¯11520
 1800 ¯7920 1800 ¯4440 ¯660 ¯8040 1800 1800 1800 ¯12420

Here, a positive number indicates an overlap between a connection and a session. A negative overlap indicates that a connection was entirely separate in time from a session.

A person may join and leave multiple times, even during a session. There is an edge case where someone may join, watch part of a session, be disconnected and reconnect to watch the rest of the session. Therefore, we must sum positive overlaps to find the total overlap between a person's connections and each of the sessions.

We will enforce a lower bound of $0$ for overlaps using the **ceiling** (max) function `⍺⌈⍵`, so that we are only adding positive contributions to the overlaps.

We then need to sum overlaps grouped by attendee.

The **key** operator `⍺ F⌸ ⍵` will apply its operand function `F` to parts of its right argument `⍵` grouped according to keys in its left argument `⍺`.

For example, if we have apples held by 5 people in 3 groups labelled A, B and C:

In [45]:
apples←2 0 1 3 6
groups←'ABBCA'

Then the number of people in each group is given by:

In [46]:
groups {⍺,≢⍵}⌸ apples

A 2
B 2
C 1

This is the same as the monadic case applied to just groups:

In [47]:
{⍺,≢⍵}⌸groups

A 2
B 2
C 1

The total number of apples held by people in each group is:

In [48]:
groups {⍺,+/⍵}⌸ apples

A 8
B 1
C 3

We will use this to add up the session-connection overlaps for each attendee.

In [49]:
names ← Attendees 'Attendee'
10 10↑ overlap ← (connected⌿names) {+⌿⍵}⌸ 0 ⌈ (leave∘.⌊end) - (join∘.⌈start)

900 2700 900 1800 1800 900 1500 1200 900 1800
900 2700 900 1800 1800 900 1500 1200 900 1800
 0 0 0 180 1800 60 0 0 0 0
900 2700 900 1740 1800 900 1500 1200 240 0
 0 2040 900 1800 1800 900 1500 1200 900 1800
900 2700 900 1800 1800 900 1500 1200 900 1800
840 2700 900 1800 1800 900 1500 1200 900 1800
960 2700 900 1800 1800 900 1500 1200 900 1800
900 2700 900 1800 1800 900 1500 1200 180 0
 0 60 0 0 0 0 0 0 0 0

According to our problem specification, a person attended a session if they were present at a session for *at least half* of the duration of that session.

The duration of a session is just the difference between its start and end times.

In [50]:
end-start

900 2700 900 1800 1800 900 1500 1200 900 1800 600 3000 2700 1800 900 600 2100 900 1800 1800 900 900 300 900 1800 1800 900 1800 1800 900 1200 1800 600 1800 1800 300 1800 1800 900 600 2100 900 1800 1800 900 1800 2700 300

To find the attendance ratio, we need to compare each session duration (single number) with each row of overlaps. We will match these up using the **bracket axis** operator. This special syntax allows us to specify a particular axis on which to apply a function.

In [51]:
⍴overlap
⍴end-start

333 48

48

There are 48 sessions, so we will specify the 2nd axis which corresponds to the number of columns (number of sessions) and has length 48.

In [52]:
10 10↑ overlap ÷[2] end-start

1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1
0 0 0 0.1 1 0.06666666667 0 0 0 0
1 1 1 0.9666666667 1 1 1 1 0.2666666667 0
0 0.7555555556 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1
0.9333333333 1 1 1 1 1 1 1 1 1
1.066666667 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 0.2 0
0 0.02222222222 0 0 0 0 0 0 0 0

Our final result is where those overlaps are greater than or equal to 0.5. However, we also want to expand our result for those people who registered but did not attend.

In [53]:
⍴overlap
⍴(names{∨⌿⍵}⌸connected) ⍀ 0.5 ≤ overlap ÷[2] end-start

333 48

435 48

It would be cheaper if we could avoid using key `⌸` a 2nd time for grouping the `connected` vector. We could avoid this by using expand on the `connections` matrix which makes our `join` and `leave` vectors, but our solution would be incorrect if there were sessions around 1st January 1970.

In [54]:
20 ¯1⎕DT 0

┌────────────────┐
│1970 1 1 0 0 0 0│
└────────────────┘

Let's put all the pieces together and see our final solution once again:

In [55]:
∇  Attended←{
     a_cols←1⌷⍺ ⋄ a_data←1↓⍺
     s_cols←1⌷⍵ ⋄ s_data←1↓⍵
     Attendees←{a_data[;a_cols⍳⊆⍵]}
     Schedule←{s_data[;s_cols⍳⊆⍵]}
     Timestamp2Unix←{20 ⎕DT ¯1(⌽@1 2 3)¨2⊃¨': /'∘⎕VFI¨⊆⍵}

     connections←Attendees'Join Time' 'Leave Time'
     connected←'--'∘≢¨connections[;1]

     (join leave)←↓⍉Timestamp2Unix connected⌿connections
     (start end)←↓⍉Timestamp2Unix Sch'Start Time' 'End Time'

     names←Att'Attendee'
     overlap←names{+⌿⍵}⌸⍉0⌈(end∘.⌊leave)-(start∘.⌈join)
     0.5≤(names{∨⌿⍵}⌸connected)⍀overlap÷[2](end-start)
 }
∇